In [1]:
import Pkg; 

if split(pwd(),"/")[end] == "utility"
    cd(joinpath(@__DIR__, "../../../MorphoMolMonteCarlo/"))
    Pkg.activate("Project.toml")
end

using MorphoMol
using StaticArrays
using Rotations
using Distances
using JLD2

if split(pwd(), "/")[end] == "MorphoMolMonteCarlo"
    cd(joinpath(@__DIR__, "../MMMCNotebooks/"))
end

  Activating project at `~/Doktor/Code/MorphoMolMonteCarlo`


In [2]:
rwm_hot_start_folder = "assets/input/jld2/2_low/"

for file in readdir("assets/input/jld2/2_low/")
    if split(file, ".")[end] == "jld2" && occursin("6r7m", file)
        @load "$rwm_hot_start_folder$file" template_mol template_radii x_init
        realization = MorphoMol.Utilities.get_flat_realization(x_init, template_mol)
        E = MorphoMol.Energies.solvation_free_energy(
            realization, 
            length(template_radii),
            [template_radii; template_radii],
            1.4,
            MorphoMol.Energies.get_prefactors(1.4, 0.3665),
            0.0,
            0.85
            )
        println(file, ": ", E)
    end
end

6r7m_1.jld2: 8143.382898458421
6r7m_10.jld2: 8141.4136689220795
6r7m_11.jld2: 8141.520791211802
6r7m_12.jld2: 8153.085424390234
6r7m_13.jld2: 8153.313369050566
6r7m_14.jld2: 8154.8717345610585
6r7m_15.jld2: 8142.208346347945
6r7m_16.jld2: 8142.64178768809
6r7m_17.jld2: 8140.4095298520315
6r7m_18.jld2: 8138.924225194982
6r7m_19.jld2: 8141.6935359423005
6r7m_2.jld2: 8142.1430598848
6r7m_20.jld2: 8141.293820963843
6r7m_21.jld2: 8146.020574241854
6r7m_3.jld2: 8139.820849722495
6r7m_4.jld2: 8141.411219690925
6r7m_5.jld2: 8139.4468587013
6r7m_6.jld2: 8152.625762472693
6r7m_7.jld2: 8152.571329720671
6r7m_8.jld2: 8141.157897383011
6r7m_9.jld2: 8140.0076230678815


In [13]:
# folder = "assets/hpc_out/hpc_2_out_refined/"
# for file in readdir(folder)
#     if split(file, ".")[end] == "jld2"
#         mt = split(file, "_")[2]
#         num = split(file, "_")[3]
#         mkpath(joinpath(folder, "$(mt)_$(num)"))
#         mv(joinpath(folder, file), joinpath(folder, "$(mt)_$(num)", file))
#     end
# end

In [3]:
min_path_by_mol = Dict("6r7m" => "", "6sae" => "", "6sag" => "", "2tmv" => "", "1ei7" => "")
E_min_by_mol = Dict("6r7m" => Inf, "6sae" => Inf, "6sag" => Inf, "2tmv" => Inf, "1ei7" => Inf)
mol_types = ["6r7m", "6sae", "6sag", "2tmv", "1ei7"]

for mol_type in mol_types
    for file in readdir(rwm_hot_start_folder)
        if  split(file, ".")[end] == "jld2" && split(file, "_")[1] == mol_type
            try
                @load "$rwm_hot_start_folder$file" template_mol template_radii x_init
                realization = MorphoMol.Utilities.get_flat_realization(x_init, template_mol)
                E = MorphoMol.Energies.solvation_free_energy(
                    realization, 
                    length(template_radii),
                    [template_radii; template_radii],
                    1.4,
                    MorphoMol.Energies.get_prefactors(1.4, 0.3665),
                    0.0,
                    0.85
                    )
                if E_min_by_mol[mol_type] > E
                    E_min_by_mol[mol_type] = E
                    min_path_by_mol[mol_type] = "$rwm_hot_start_folder/$file"
                end
            catch e
                println(e)
            end
        end
    end
end

for mol_type in mol_types
    if min_path_by_mol[mol_type] != ""
        @load min_path_by_mol[mol_type] template_mol template_radii x_init
        realization = MorphoMol.Utilities.get_flat_realization(x_init, template_mol)
        E = MorphoMol.Energies.solvation_free_energy(
            realization, 
            length(template_radii),
            [template_radii; template_radii],
            1.4,
            MorphoMol.Energies.get_prefactors(1.4, 0.3665),
            0.0,
            0.85
            )
        template_mol_sim = deepcopy(template_mol)
        x_sim = deepcopy(x_init)
        @load "assets/input/jld2/2_experimental/2_$(mol_type)_protor.jld2" template_mol template_radii x_init
        println(mol_type, " ", E, " ", MorphoMol.Utilities.average_offset_distance(template_mol_sim, template_mol, x_sim, x_init))
    end
end


6r7m 8138.924225194982 0.9982893464282008
6sae 8107.279622532144 1.2821182928454404
6sag 8097.730817128498 1.1146356769067605
2tmv 8056.550361360152 21.083570792125695
1ei7 8185.798536861604 24.19202036075877


In [4]:
folder = "assets/hpc_out_2_refined/"

E_min_by_mol = Dict("6r7m" => Inf, "6sae" => Inf, "6sag" => Inf, "2tmv" => Inf, "1ei7" => Inf)
min_path_by_mol = Dict("6r7m" => "", "6sae" => "", "6sag" => "", "2tmv" => "", "1ei7" => "")
for mol_type in mol_types
    E_min = E_min_by_mol[mol_type]
    min_file_path = ""
    for (root, dirs, _) in walkdir(folder)
        for dir in dirs
            for file in readdir("$(folder)$(dir)/")
                if  split(file, ".")[end] == "jld2" && occursin(mol_type, file)
                    try
                        @load "$(folder)$(dir)/$file" in_out_data
                        if E_min > minimum(in_out_data.output.Es)
                            E_min = minimum(in_out_data.output.Es)
                            min_file_path = "$(folder)$(dir)/$file"
                        end
                    catch e
                        println(e)
                    end
                end
            end
        end
    end
    min_path_by_mol[mol_type] = min_file_path
end

┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/twZ5D/src/JLD2.jl:298


EOFError()


In [5]:
println(min_path_by_mol)

Dict("6sag" => "assets/hpc_out_2_refined/6sag_5/3_6sag_5_protor_oj_0_0_os_0_85.jld2", "1ei7" => "assets/hpc_out_2_refined/_2_low_refinement_a/1ei7_20_protor_oj_0_0_os_0_85.jld2", "6r7m" => "assets/hpc_out_2_refined/_2_low_refinement_d/6r7m_2_protor_oj_0_0_os_0_85.jld2", "6sae" => "assets/hpc_out_2_refined/6sae_13/3_6sae_13_protor_oj_0_0_os_0_85.jld2", "2tmv" => "assets/hpc_out_2_refined/_2_low_refinement_a/2tmv_14_protor_oj_0_0_os_0_85.jld2")


In [6]:
for mol_type in ["6r7m", "6sae", "6sag", "2tmv", "1ei7"]
    if min_path_by_mol[mol_type] != ""
        @load min_path_by_mol[mol_type] in_out_data
        template_mol_sim = in_out_data.input.template_mol
        x_sim = in_out_data.output.states[1]
        
        @load "assets/input/jld2/2_experimental/2_$(mol_type)_protor.jld2" template_mol template_radii x_init
        println(mol_type, " ", minimum(in_out_data.output.Es), " ", MorphoMol.Utilities.average_offset_distance(template_mol_sim, template_mol, x_sim, x_init))
    end
end 

6r7m 8135.206810221899 1.1282993528234022
6sae 8106.102486349152 1.1429573909935333
6sag 8092.188608572787 1.1146356769067605
2tmv 8055.0066341330785 21.058481312588814
1ei7 8184.265920109759 0.9825672375691238


In [7]:
energies_by_mol_type = Dict("6r7m" => [], "6sae" => [], "6sag" => [], "2tmv" => [], "1ei7" => [])
for (root, dirs, _) in walkdir("assets/hpc_out_2_refined/")
    for dir in root .* dirs
        for file in readdir(dir)
            if split(file, ".")[end] == "jld2"
                try
                    @load "$(dir)/$(file)" in_out_data
                    push!(energies_by_mol_type[split(file, "_")[2]], (minimum(in_out_data.output.Es), "$(dir)/$(file)"))
                catch e
                    println(e)
                end
            end
        end
    end
end

EOFError()


┌ Warning: Opening file with JLD2.MmapIO failed, falling back to IOStream
└ @ JLD2 ~/.julia/packages/JLD2/twZ5D/src/JLD2.jl:298


KeyError("17")
KeyError("18")
KeyError("19")
KeyError("20")
KeyError("21")
KeyError("4")
KeyError("5")
KeyError("7")
KeyError("8")
KeyError("9")
KeyError("10")
KeyError("11")
KeyError("12")
KeyError("13")
KeyError("14")
KeyError("15")
KeyError("16")
KeyError("17")
KeyError("1")
KeyError("2")
KeyError("3")
KeyError("6")
KeyError("7")
KeyError("8")
KeyError("9")
KeyError("10")
KeyError("12")
KeyError("13")
KeyError("14")
KeyError("16")
KeyError("17")
KeyError("18")
KeyError("19")
KeyError("1")
KeyError("2")
KeyError("5")
KeyError("6")
KeyError("7")
KeyError("8")
KeyError("9")
KeyError("10")
KeyError("11")
KeyError("12")
KeyError("13")
KeyError("1")
KeyError("2")
KeyError("3")
KeyError("4")
KeyError("5")
KeyError("6")
KeyError("7")
KeyError("8")
KeyError("9")
KeyError("1")
KeyError("2")
KeyError("3")
KeyError("4")
KeyError("5")
KeyError("6")
KeyError("17")
KeyError("18")
KeyError("19")
KeyError("20")
KeyError("21")
KeyError("4")
KeyError("5")
KeyError("7")
KeyError("8")
KeyError("9")
KeyE

In [ ]:
for mol_type in ["6sae", "6sag", "2tmv", "1ei7"]
    mkpath("assets/output/2_low_min_polys/$(mol_type)")
    for (i,elem) in enumerate(sort!(energies_by_mol_type[mol_type])[1:end])
        @load "$(elem[2])" in_out_data
        mindex = argmin(in_out_data.output.Es)
        state = in_out_data.output.states[mindex]
        radii = collect(Base.Iterators.flatten([in_out_data.input.template_radii for _ in 1:in_out_data.input.number_of_molecules]))
        MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(state, in_out_data.input.template_mol), radii, "assets/output/2_low_min_polys/$(mol_type)/$(i)", in_out_data.input.number_of_molecules, length(in_out_data.input.template_radii))
    end
end

dir = "assets/input/jld2/2_low/"
mkpath("assets/output/2_low_min_polys/6r7m/")
i = 1
for file in readdir(dir)
    if split(file, ".")[end] == "jld2" && occursin("6r7m", file)
        try
            @load "$(dir)$(file)" template_mol template_radii x_init
            n_mol = Int(length(x_init) / 6)
            n_atoms_per_mol = length(template_radii)
            radii = [template_radii; template_radii]
            MorphoMolNotebooks.configuration_to_poly(MorphoMol.Utilities.get_flat_realization(x_init, template_mol), radii, "assets/output/2_low_min_polys/6r7m/$(i)", 2, length(template_radii))
            i += 1
        catch e
            println(e)
        end
    end
end


In [8]:
E_disps = Dict("6r7m" => 0.0, "6sae" => 0.0, "6sag" => 0.0, "2tmv" => 0.0, "1ei7" => 0.0)
folder = "assets/input/poly/singles/"
for file in readdir(folder)
    c,r = MorphoMol.Utilities.poly_to_state("$folder$file")
    E = MorphoMol.Energies.solvation_free_energy(
        c, 
        length(r),
        r,
        1.4,
        MorphoMol.Energies.get_prefactors(1.4, 0.3665),
        0.0,
        0.85
        )
    E_disps[split(file, "_")[1]] = 2*E
end

In [11]:
dir = "assets/input/poly/2_low_1_2/"
for file in readdir(dir)
    realization, radii = MorphoMol.Utilities.poly_to_state("$(dir)$(file)")
    E = MorphoMol.Energies.solvation_free_energy(
            realization, 
            Int(length(radii) / 2),
            radii,
            1.4,
            MorphoMol.Energies.get_prefactors(1.4, 0.3665),
            0.0,
            0.85
            )
    id = split(file, "_")[1]
    place = split(file, "_")[2]
    println("$(place) $(id): $(E - E_disps[id])")
end

1.poly 1ei7: -70.72023364027973
2.poly 1ei7: -70.66959505642535
1.poly 2tmv: -75.03710864535515
2.poly 2tmv: -68.49805296157228
1.poly 6r7m: -76.19540578103351
2.poly 6r7m: -66.49287984922012
1.poly 6sae: -79.1152379615969
2.poly 6sae: -72.72268371966675
1.poly 6sag: -81.9693275475529
2.poly 6sag: -69.4876095604277
